# --- Day 3: Spiral Memory ---

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```

While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square 1.

For example:

Data from square 1 is carried 0 steps, since it's at the access port.
Data from square 12 is carried 3 steps, such as: down, left, left.
Data from square 23 is carried only 2 steps: up twice.
Data from square 1024 must be carried 31 steps.
How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?

Your puzzle input is `312051`.

From eyeballing the data square, it makes sense to only deal with odd sized squares, so everytime we make the next size square we start with the bottom right corner:

In [7]:
def find_square_size(n):
    '''given a number n, requrn the side length of the odd square it fits in'''
    
    # get square root and round up
    num = int(np.ceil(np.sqrt(n))) 
    
    # make sure the square we're ruturning is a odd square
    while num % 2 == 0:
        num += 1        
    return num

[find_square_size(n) for n in [1,2,3,4,5,6,7,8,9,10,11,49, 1000]]

[1, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 7, 33]

# simple solution

We know the square the number is in, so by starting with the one smaller odd sized square, as we build that up the number has to be in one of the 4 sides.

In [78]:
def spiral_pos(n):    
    big_square = find_square_size(n)
    small_square = big_square - 2

    side_length = (big_square ** 2 - small_square **2)//4
    mid = (small_square // 2)
    cur_num = small_square**2
    
    if n == small_square**2:
        return mid, -mid, mid*2
    else:
        mid += 1
        
    if n == big_square**2:
            return mid, -mid, mid*2

    for dir in "ENWS":
        if n in range(cur_num, cur_num + side_length):
            #print("Eureka!", dir, cur_num, side_length, n)
            return calc_pos(dir, cur_num, n, mid)
        else:
            cur_num += side_length
        
def calc_pos(dir, cur_num, n, mid):
    if dir == "N":
        x, y = mid, mid
        x -= n - cur_num
    elif dir == "S":
        x, y = -mid, -mid
        x += n - cur_num
    elif dir == "E":
        x, y = mid, -mid
        y += n - cur_num
    elif dir == "W":
        x, y = -mid, mid
        y -= n - cur_num
    distance = abs(x) + abs(y)
    return x, y , distance
        
spiral_pos(25)

(2, -2, 4)

In [79]:
tests = [(1, 0), 
         (12, 3), 
         (23, 2), 
         (1024, 31), 
         ]
for data_square, steps in tests:
    print(spiral_pos(data_square))

(-1, 1, -2)
(2, 1, 3)
(0, -2, 2)
(-15, 16, 31)


In [80]:
puzzle_input = 312051
spiral_pos(puzzle_input)

(-151, -279, 430)

# --- Part Two ---
As a stress test on the system, the programs here clear the grid and then store the value 1 in square 1. Then, in the same allocation order as shown above, they store the sum of the values in all adjacent squares, including diagonals.

So, the first few squares' values are chosen as follows:

- Square 1 starts with the value 1.
- Square 2 has only one adjacent filled square (with value 1), so it also stores 1.
- Square 3 has both of the above squares as neighbors and stores the sum of their values, 2.
- Square 4 has all three of the aforementioned squares as neighbors and stores the sum of their values, 4.
- Square 5 only has the first and fourth squares as neighbors, so it gets the value 5.

Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:

```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
```

What is the first value written that is larger than your puzzle input?

So, I should be able to modify the above function and store each coordinates value in a grid

In [81]:
from collections import defaultdict

In [115]:
def get_pos(n):
    if n == 1:
        return 0,0
    
    big_square = find_square_size(n)
    small_square = big_square - 2

    side_length = (big_square ** 2 - small_square **2)//4
    mid = (small_square // 2)
    cur_num = small_square**2
    
    if n == small_square**2:
        return mid, -mid
    else:
        mid += 1
        
    if n == big_square**2:
            return mid, -mid

    for dir in "ENWS":
        if n in range(cur_num, cur_num + side_length):
            if dir == "N":
                x, y = mid, mid
                x -= n - cur_num
            elif dir == "S":
                x, y = -mid, -mid
                x += n - cur_num
            elif dir == "E":
                x, y = mid, -mid
                y += n - cur_num
            elif dir == "W":
                x, y = -mid, mid
                y -= n - cur_num
            return x, y
        else:
            cur_num += side_length
        

def spiral_vals(n):
    
    vals = defaultdict(int)
    vals[(0,0)] = 1
    
    for i in range(2,n):
        value = 0
        pos = get_pos(i)
        for neighbour in get_neighbours(i):
            value += vals[neighbour]
        vals[pos] = value
        if value > n:
            return value
    print(vals)
    
spiral_vals(puzzle_input)

312453

In [113]:
def get_neighbours(n):
    """takes in a num 9 and returns a list of its neighbours"""
    neighbours = []
    x, y = get_pos(n)
    for xx in [x-1, x, x+1]:
        for yy in [y-1, y, y+1]:
            neighbours.append((xx,yy))
    neighbours.remove((x,y))
    return neighbours

get_neighbours(500)

[(-12, -5),
 (-12, -4),
 (-12, -3),
 (-11, -5),
 (-11, -3),
 (-10, -5),
 (-10, -4),
 (-10, -3)]